In [1]:
import pandas as pd
import numpy as np
import glob

In [175]:
path = './'
#/model-dev-coding-exercise-public-master'
contracts_path = path + '/contracts'
fuel_prices_path = path + '/fuelPrices'
plant_parameters_path = path + '/plantParameters'
power_prices_path = path + '/powerPrices'

output_path = path + '/output'

In [13]:
def ConcatCSV(dir_path):
    all_files = glob.glob(dir_path + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col = 'Date', parse_dates= True)
        li.append(df)
    data = pd.concat(li, axis = 0)
    return data

In [253]:
contracts = pd.read_csv(contracts_path +"/Contracts.csv")
plant_parameters = pd.read_csv(plant_parameters_path+ "/Plant_Parameters.csv")
fuel_prices = ConcatCSV(fuel_prices_path)
power_prices = ConcatCSV(power_prices_path)

In [17]:
contracts.head()

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,2017-03-31,20000,Daily,3.0,NaN,Henry Hub
1,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
2,O1,European option,2017-06-01,2017-07-15,10000,Daily,2.9,0.1,GDA_TETSTX
3,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [18]:
plant_parameters.head()

,PlantName,Year,Month,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost
0,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.1,10000
1,Alpha,2017,2,HB_HOUSTON,Henry Hub,98,9.4,3,0.1,10000
2,Alpha,2017,3,HB_HOUSTON,Henry Hub,96,9.6,3,0.1,10000
3,Alpha,2017,4,HB_HOUSTON,Henry Hub,94,9.8,3,0.1,10000
4,Alpha,2017,5,HB_HOUSTON,Henry Hub,92,10.0,3,0.1,10000


In [21]:
fuel_prices.head()

,Variable,Price
Date,,
2019-05-01,GDA_TETSTX,2.490
2019-04-30,GDA_TETSTX,2.420
2019-04-29,GDA_TETSTX,2.445
2019-04-28,GDA_TETSTX,2.445
2019-04-27,GDA_TETSTX,2.445


In [72]:
power_prices.head()

,SettlementPoint,Price
Date,,
2016-01-01,HB_BUSAVG,18.42
2016-01-01,HB_HOUSTON,18.42
2016-01-01,HB_HUBAVG,18.42
2016-01-01,HB_NORTH,18.43
2016-01-01,HB_SOUTH,18.38


## Basic Stats

In [73]:
def GetBasicStats(df):
    basic_stats = df.groupby(by=['SettlementPoint', df.index.year, df.index.month]).agg({"Price": [np.mean, min, max, np.std]})
    basic_stats.rename_axis(index = ['SettlementPoint', 'Year', 'Month'], inplace = True)
    basic_stats.reset_index(inplace = True)
    basic_stats.columns = basic_stats.columns.droplevel(0)
    basic_stats.set_axis(['SettlementPoint',  'Year', 'Month', 'Mean', 'Min', 'Max', 'SD'], axis='columns', inplace=True)
    return basic_stats

In [74]:
basic_stats = GetBasicStats(power_prices)

In [75]:
basic_stats.head()

,SettlementPoint,Year,Month,Mean,Min,Max,SD
0,HB_BUSAVG,2016,1,19.209301,5.12,54.69,5.485210
1,HB_BUSAVG,2016,2,15.288247,1.79,50.98,5.777172
2,HB_BUSAVG,2016,3,16.755626,1.22,64.31,7.368497
3,HB_BUSAVG,2016,4,18.289347,1.18,93.09,9.506143
4,HB_BUSAVG,2016,5,18.159583,3.03,73.61,8.017221


In [76]:
def CalcVolatility(df):
    log_ret  = np.log(df.Price) - np.log(df.Price.shift(1))
    return np.std(log_ret)

In [77]:
def CalcHourlyVolatilityByMonth(df):
    prices = df[(df['Price']>0)]
    volatility = prices.groupby(by = ['SettlementPoint', prices.index.year, prices.index.month]).apply(CalcVolatility)
    volatility.rename_axis(index = ['SettlementPoint', 'Year', 'Month'], inplace = True)
    vol = volatility.to_frame('Volatility').reset_index()
    return vol

In [78]:
vol = CalcHourlyVolatilityByMonth(power_prices)

In [79]:
stats = pd.merge(basic_stats, vol,  how='left', left_on=['SettlementPoint','Year', 'Month'], right_on = ['SettlementPoint','Year', 'Month'])

In [80]:
stats.head()

,SettlementPoint,Year,Month,Mean,Min,Max,SD,Volatility
0,HB_BUSAVG,2016,1,19.209301,5.12,54.69,5.485210,0.162491
1,HB_BUSAVG,2016,2,15.288247,1.79,50.98,5.777172,0.222087
2,HB_BUSAVG,2016,3,16.755626,1.22,64.31,7.368497,0.212648
3,HB_BUSAVG,2016,4,18.289347,1.18,93.09,9.506143,0.228768
4,HB_BUSAVG,2016,5,18.159583,3.03,73.61,8.017221,0.172522


In [82]:
stats.to_csv(output_path + '/MonthlyPowerPriceStatistics.csv', index = False)

In [83]:
contracts

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,2017-03-31,20000,Daily,3.0,NaN,Henry Hub
1,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
2,O1,European option,2017-06-01,2017-07-15,10000,Daily,2.9,0.1,GDA_TETSTX
3,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [87]:
daily_contracts = contracts[contracts.Granularity == 'Daily']
hourly_contracts = contracts[contracts.Granularity == 'Hourly']

In [88]:
hourly_contracts

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
1,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
3,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [94]:
def GetExpandedContracts(df, freq):
    return pd.concat([pd.DataFrame({'Start': pd.date_range(row.StartDate, row.EndDate, freq=freq),
               'ContractName': row.ContractName,
               'DealType': row.DealType,
               'Volume': row.Volume,
               'Granularity': row.Granularity,
               'StrikePrice': row.StrikePrice,
               'Premium': row.Premium,
               'PriceName': row.PriceName,}, columns=['Start', 'ContractName', 'DealType', 'Volume', 'Granularity', 'StrikePrice', 'Premium', 'PriceName']) 
           for i, row in df.iterrows()], ignore_index=True)

In [97]:
daily_expanded =  GetExpandedContracts(daily_contracts, 'D')

In [98]:
hourly_expanded = GetExpandedContracts(hourly_contracts, 'H')

In [104]:
fuel_prices.tail()

,Variable,Price
Date,,
2020-09-26,Henry Hub,1.895
2020-09-27,Henry Hub,1.895
2020-09-28,Henry Hub,1.895
2020-09-29,Henry Hub,1.835
2020-09-30,Henry Hub,1.690


In [113]:
fuel_prices = fuel_prices.reset_index()

In [114]:
daily_expanded.head()

,Start,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName
0,2017-01-01,S1,Swap,20000,Daily,3.0,NaN,Henry Hub
1,2017-01-02,S1,Swap,20000,Daily,3.0,NaN,Henry Hub
2,2017-01-03,S1,Swap,20000,Daily,3.0,NaN,Henry Hub
3,2017-01-04,S1,Swap,20000,Daily,3.0,NaN,Henry Hub
4,2017-01-05,S1,Swap,20000,Daily,3.0,NaN,Henry Hub


In [115]:
daily_prices = pd.merge(daily_expanded, fuel_prices,  how='left', left_on=['Start','PriceName'], right_on = ['Date','Variable'])

In [118]:
power_prices.reset_index(inplace=True)

In [119]:
power_prices.head()

,Date,SettlementPoint,Price
0,2016-01-01,HB_BUSAVG,18.42
1,2016-01-01,HB_HOUSTON,18.42
2,2016-01-01,HB_HUBAVG,18.42
3,2016-01-01,HB_NORTH,18.43
4,2016-01-01,HB_SOUTH,18.38


In [120]:
hourly_prices = pd.merge(hourly_expanded, power_prices,  how='left', left_on=['Start','PriceName'], right_on = ['Date','SettlementPoint'])

In [121]:
hourly_prices

,Start,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date,SettlementPoint,Price
0,2017-01-01 00:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 00:00:00,HB_NORTH,23.23
1,2017-01-01 01:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 01:00:00,HB_NORTH,21.82
2,2017-01-01 02:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 02:00:00,HB_NORTH,20.16
3,2017-01-01 03:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 03:00:00,HB_NORTH,20.00
4,2017-01-01 04:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 04:00:00,HB_NORTH,20.24
5,2017-01-01 05:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 05:00:00,HB_NORTH,21.45
6,2017-01-01 06:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 06:00:00,HB_NORTH,24.35
7,2017-01-01 07:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 07:00:00,HB_NORTH,24.06
8,2017-01-01 08:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 08:00:00,HB_NORTH,25.48
9,2017-01-01 09:00:00,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 09:00:00,HB_NORTH,24.94


In [149]:
def CalculatePayoff(df):
    filt = df['DealType'] == 'Swap'
    df.loc[filt, 'Payoff'] = (df[filt].Price - df[filt].StrikePrice)*df[filt].Volume
    
    opt_filt = df['DealType'] == 'European option'
    df.loc[opt_filt, 'Payoff'] = ((df[opt_filt].Price - df[opt_filt].StrikePrice).clip(0, None) - df[opt_filt].Premium)*df[opt_filt].Volume
    
    return df

In [151]:
daily_payoffs = CalculatePayoff(daily_prices)

In [152]:
hourly_payoffs = CalculatePayoff(hourly_prices)

In [159]:
daily_total_payoffs =  daily_prices.groupby(by = ['ContractName', daily_prices.Date.dt.year, daily_prices.Date.dt.month])['Payoff'].sum()

In [163]:
daily_total_payoffs.rename_axis(index = ['ContractName', 'Year', 'Month'], inplace = True)
total_payoffs = daily_total_payoffs.to_frame('TotalPayoff').reset_index()

In [164]:
total_payoffs

,ContractName,Year,Month,TotalPayoff
0,O1,2017,6,-20100.0
1,O1,2017,7,-14300.0
2,S1,2017,1,198300.0
3,S1,2017,2,-97700.0
4,S1,2017,3,-97700.0


In [165]:
def CalculateTotalPayoffs(df):
    payoffs =  df.groupby(by = ['ContractName', df.Date.dt.year, df.Date.dt.month])['Payoff'].sum()
    payoffs.rename_axis(index = ['ContractName', 'Year', 'Month'], inplace = True)
    total_payoffs = payoffs.to_frame('TotalPayoff').reset_index()
    return total_payoffs

In [168]:
daily_total_payoffs = CalculateTotalPayoffs(daily_prices)
hourly_total_payoffs = CalculateTotalPayoffs(hourly_prices)

In [170]:
total_payoffs = pd.concat([daily_total_payoffs, hourly_total_payoffs])

In [172]:
total_payoffs.to_csv(output_path + '/MonthlyContractPayoffs.csv', index= False)

In [188]:
plant_parameters.dtypes

PlantName                  object
Year                        int64
Month                       int64
PowerPriceName             object
FuelPriceName              object
Capacity                    int64
HeatRate                  float64
VOM                         int64
FuelTransportationCost    float64
FixedStartCost              int64
dtype: object

In [254]:
fuel_pr = fuel_prices

In [255]:
fuel_pr.reset_index(inplace = True)
fuel_pr['date'] = fuel_pr.Date.dt.date
fuel_pr['Month'] = fuel_pr.Date.dt.month
fuel_pr['Year'] = fuel_pr.Date.dt.year

In [256]:
fuel_pr.head()

,Date,Variable,Price,date,Month,Year
0,2019-05-01,GDA_TETSTX,2.490,2019-05-01,5,2019
1,2019-04-30,GDA_TETSTX,2.420,2019-04-30,4,2019
2,2019-04-29,GDA_TETSTX,2.445,2019-04-29,4,2019
3,2019-04-28,GDA_TETSTX,2.445,2019-04-28,4,2019
4,2019-04-27,GDA_TETSTX,2.445,2019-04-27,4,2019


In [257]:
fuel_prices.head()

,Date,Variable,Price,date,Month,Year
0,2019-05-01,GDA_TETSTX,2.490,2019-05-01,5,2019
1,2019-04-30,GDA_TETSTX,2.420,2019-04-30,4,2019
2,2019-04-29,GDA_TETSTX,2.445,2019-04-29,4,2019
3,2019-04-28,GDA_TETSTX,2.445,2019-04-28,4,2019
4,2019-04-27,GDA_TETSTX,2.445,2019-04-27,4,2019


In [258]:
daily_running_costs = pd.merge(fuel_prices, plant_parameters,  how='right', left_on=['Variable', 'Month', 'Year'], right_on = ['FuelPriceName', 'Month', 'Year'])

In [259]:
daily_running_costs['RunningCost'] = ((daily_running_costs['Price']+ daily_running_costs['FuelTransportationCost'])*daily_running_costs['HeatRate']) + daily_running_costs['VOM']

In [260]:
plant_parameters.head()

,PlantName,Year,Month,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost
0,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.1,10000
1,Alpha,2017,2,HB_HOUSTON,Henry Hub,98,9.4,3,0.1,10000
2,Alpha,2017,3,HB_HOUSTON,Henry Hub,96,9.6,3,0.1,10000
3,Alpha,2017,4,HB_HOUSTON,Henry Hub,94,9.8,3,0.1,10000
4,Alpha,2017,5,HB_HOUSTON,Henry Hub,92,10.0,3,0.1,10000


In [261]:
power_prices.head()

,SettlementPoint,Price
Date,,
2016-01-01,HB_BUSAVG,18.42
2016-01-01,HB_HOUSTON,18.42
2016-01-01,HB_HUBAVG,18.42
2016-01-01,HB_NORTH,18.43
2016-01-01,HB_SOUTH,18.38


In [262]:
power_prices.reset_index(inplace = True)
power_prices['date'] = power_prices.Date.dt.date
#power_prices['Year'] = power_prices.Date.dt.year

In [263]:
power_prices.head()

,Date,SettlementPoint,Price,date
0,2016-01-01,HB_BUSAVG,18.42,2016-01-01
1,2016-01-01,HB_HOUSTON,18.42,2016-01-01
2,2016-01-01,HB_HUBAVG,18.42,2016-01-01
3,2016-01-01,HB_NORTH,18.43,2016-01-01
4,2016-01-01,HB_SOUTH,18.38,2016-01-01


In [264]:
daily_running_costs.head()

,Date,Variable,Price,date,Month,Year,PlantName,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,RunningCost
0,2018-12-31,GDA_TETSTX,3.080,2018-12-31,12,2018,Bravo,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,25.788
1,2018-12-30,GDA_TETSTX,3.080,2018-12-30,12,2018,Bravo,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,25.788
2,2018-12-29,GDA_TETSTX,3.080,2018-12-29,12,2018,Bravo,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,25.788
3,2018-12-28,GDA_TETSTX,3.080,2018-12-28,12,2018,Bravo,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,25.788
4,2018-12-27,GDA_TETSTX,2.915,2018-12-27,12,2018,Bravo,HB_SOUTH,GDA_TETSTX,250,7.6,2,0.05,50000,24.534


In [225]:
daily_running_costs.dtypes

level_0                            int64
index                              int64
Date                      datetime64[ns]
Variable                          object
Price                            float64
Month                              int64
Year                               int64
PlantName                         object
PowerPriceName                    object
FuelPriceName                     object
Capacity                           int64
HeatRate                         float64
VOM                                int64
FuelTransportationCost           float64
FixedStartCost                     int64
RunningCost                      float64
dtype: object

In [265]:
hourly_running_costs = pd.merge(power_prices, daily_running_costs,  how='right', left_on=['SettlementPoint', 'date'], right_on = ['PowerPriceName', 'date'])

In [266]:
hourly_running_costs

,Date_x,SettlementPoint,Price_x,date,Date_y,Variable,Price_y,Month,Year,PlantName,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,RunningCost
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
1,2017-01-01 01:00:00,HB_HOUSTON,21.82,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
2,2017-01-01 02:00:00,HB_HOUSTON,20.16,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
3,2017-01-01 03:00:00,HB_HOUSTON,20.00,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
4,2017-01-01 04:00:00,HB_HOUSTON,20.24,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
5,2017-01-01 05:00:00,HB_HOUSTON,21.45,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
6,2017-01-01 06:00:00,HB_HOUSTON,24.35,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
7,2017-01-01 07:00:00,HB_HOUSTON,24.06,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
8,2017-01-01 08:00:00,HB_HOUSTON,25.48,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
9,2017-01-01 09:00:00,HB_HOUSTON,24.94,2017-01-01,2017-01-01,Henry Hub,3.650,1,2017,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500


In [268]:
hourly_running_costs.drop(['date', 'Date_y', 'Month', 'Year'], inplace= True, axis=1)

In [270]:
hourly_running_costs.rename(columns={'Date_x':'Date', 'Price_x':'PowerPrice', 'Price_y':'FuelPrice'}, inplace=True)

In [271]:
hourly_running_costs

,Date,SettlementPoint,PowerPrice,Variable,FuelPrice,PlantName,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,RunningCost
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
1,2017-01-01 01:00:00,HB_HOUSTON,21.82,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
2,2017-01-01 02:00:00,HB_HOUSTON,20.16,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
3,2017-01-01 03:00:00,HB_HOUSTON,20.00,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
4,2017-01-01 04:00:00,HB_HOUSTON,20.24,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
5,2017-01-01 05:00:00,HB_HOUSTON,21.45,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
6,2017-01-01 06:00:00,HB_HOUSTON,24.35,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
7,2017-01-01 07:00:00,HB_HOUSTON,24.06,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
8,2017-01-01 08:00:00,HB_HOUSTON,25.48,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500
9,2017-01-01 09:00:00,HB_HOUSTON,24.94,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500


In [273]:
hourly_running_costs.loc[(hourly_running_costs['PowerPrice'] > hourly_running_costs['RunningCost']), 'Generation'] = hourly_running_costs['Capacity']
hourly_running_costs.loc[(hourly_running_costs['PowerPrice'] <= hourly_running_costs['RunningCost']), 'Generation'] = 0

In [274]:
hourly_running_costs

,Date,SettlementPoint,PowerPrice,Variable,FuelPrice,PlantName,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,RunningCost,Generation
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
1,2017-01-01 01:00:00,HB_HOUSTON,21.82,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
2,2017-01-01 02:00:00,HB_HOUSTON,20.16,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
3,2017-01-01 03:00:00,HB_HOUSTON,20.00,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
4,2017-01-01 04:00:00,HB_HOUSTON,20.24,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
5,2017-01-01 05:00:00,HB_HOUSTON,21.45,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
6,2017-01-01 06:00:00,HB_HOUSTON,24.35,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
7,2017-01-01 07:00:00,HB_HOUSTON,24.06,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
8,2017-01-01 08:00:00,HB_HOUSTON,25.48,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0
9,2017-01-01 09:00:00,HB_HOUSTON,24.94,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0


In [275]:
hourly_running_costs['RunningMargin'] = (hourly_running_costs['PowerPrice'] - hourly_running_costs['RunningCost'])*hourly_running_costs['Generation']

In [276]:
hourly_running_costs

,Date,SettlementPoint,PowerPrice,Variable,FuelPrice,PlantName,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,RunningCost,Generation,RunningMargin
0,2017-01-01 00:00:00,HB_HOUSTON,23.23,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
1,2017-01-01 01:00:00,HB_HOUSTON,21.82,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
2,2017-01-01 02:00:00,HB_HOUSTON,20.16,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
3,2017-01-01 03:00:00,HB_HOUSTON,20.00,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
4,2017-01-01 04:00:00,HB_HOUSTON,20.24,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
5,2017-01-01 05:00:00,HB_HOUSTON,21.45,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
6,2017-01-01 06:00:00,HB_HOUSTON,24.35,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
7,2017-01-01 07:00:00,HB_HOUSTON,24.06,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
8,2017-01-01 08:00:00,HB_HOUSTON,25.48,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0
9,2017-01-01 09:00:00,HB_HOUSTON,24.94,Henry Hub,3.650,Alpha,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000,37.500,0.0,-0.0


In [282]:
alpha = hourly_running_costs[hourly_running_costs.PlantName == 'Alpha']

In [283]:
running_margin = alpha.RunningMargin

In [284]:
running_margin

0         -0.0
1         -0.0
2         -0.0
3         -0.0
4         -0.0
5         -0.0
6         -0.0
7         -0.0
8         -0.0
9         -0.0
10        -0.0
11        -0.0
12        -0.0
13        -0.0
14        -0.0
15        -0.0
16        -0.0
17       449.0
18        -0.0
19        -0.0
20        -0.0
21        -0.0
22        -0.0
23        -0.0
48        -0.0
49        -0.0
50        -0.0
51        -0.0
52        -0.0
53        -0.0
         ...  
34958    655.2
34959     -0.0
34960     -0.0
34961     -0.0
34962     -0.0
34963     -0.0
34988     -0.0
34989     -0.0
34990     -0.0
34991     -0.0
34992     -0.0
34993     -0.0
34994     -0.0
34995     -0.0
34996     -0.0
34997     -0.0
34998     -0.0
34999     -0.0
35000     -0.0
35001     -0.0
35002     -0.0
35003     -0.0
35004     -0.0
35005    130.2
35006     -0.0
35007     -0.0
35008     -0.0
35009     -0.0
35010     -0.0
35011     -0.0
Name: RunningMargin, Length: 17518, dtype: float64